In [1]:
import numpy as np
import pandas as pd
from nltk import word_tokenize, sent_tokenize
from string import punctuation
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Haitham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Haitham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
train = pd.read_csv('liar/raw_data/liar/train.tsv',sep='\t',header=None, names=columns)
# print(len(train))#,train.isna().sum())
# train.dropna(inplace=True)
train['label'] = train['label'].map(label_map)
train.drop(index=train[train.subject.isna()].index, inplace=True)
train.drop(index=train[train.speaker.isna()].index, inplace=True)
train.drop(index=train[train.text==' '].index, inplace=True)
train.drop(index=train[train.text=='  '].index, inplace=True)
train.drop(index=train[train.text=='\n'].index, inplace=True)
train.drop(columns=['context'],inplace=True)
train = train.reset_index()
train.count()

NameError: name 'columns' is not defined

In [5]:
train = pd.read_csv('train.csv')


In [6]:
train.count()

Statement    10240
Label        10240
dtype: int64

In [7]:
train_text = np.array(train['Statement'])

In [11]:
test = pd.read_csv('test_1.csv')


NameError: name 'preprocessing' is not defined

In [14]:
StopWords = stopwords.words('english')

def preprocessing(raw_text):
    # print(raw_text)
    tokenized = []

    
    for sent in raw_text: 
        if type(sent) is str:
          texts = [''.join([c for c in text.lower() if c not in punctuation]) for text in sent]
          texts = ''.join(texts)
          texts = [''.join([c for c in text.lower() if c not in '’']) for text in texts]
          texts = ''.join(texts)
          texts = [''.join([c for c in text.lower() if c not in '‘']) for text in texts]
          texts = ''.join(texts)
          texts = [word for word in word_tokenize(texts)] # if word not in stopWords]
          # print(texts)
          tokenized.append(texts)
    # texts = ' '.join(texts)
    # print(texts)
    return tokenized

In [15]:
test_text = np.array(test['Statement'])
train_tokens = preprocessing(train_text)
test_tokens = preprocessing(test_text)

In [16]:
count = [len(train_tokens[i]) for i in range(len(train_tokens))]
pd.DataFrame(count).describe()

0
count  10240.000000
mean      17.945410
std        9.609529
min        2.000000
25%       12.000000
50%       17.000000
75%       22.000000
max      464.000000

In [17]:
def getVocab(text, vocab):
    for txt in text:
        for w in txt:
            vocab.add(w)
    return vocab
def wordVec(text, vocab):
    word_dict = {}
    ind = 0
    for word in vocab:
        word_dict[word] = ind
        ind += 1
    word_vector = []
    for txt in text:
        w_vec = []
        for word in txt:
            w_vec.append(word_dict[word])
        word_vector.append(w_vec)
    return word_vector

In [19]:
vocab = set()
vocab = getVocab(train_tokens, vocab)
vocab = getVocab(test_tokens, vocab)
vocab = list(vocab)
vocab.sort()
len(vocab)

14959

In [21]:
# words
## convert to vectors 
word_seq_train = wordVec(train_tokens, vocab)
word_seq_test = wordVec(test_tokens, vocab)


In [22]:
pd.DataFrame(word_seq_train).to_csv('word_seq_train.csv', index=False)
pd.DataFrame(word_seq_test).to_csv('word_seq_test.csv', index=False)


In [25]:
train_label = np.array(train['Label'])
test_label = np.array(test['Label'])


pd.DataFrame(train_label,columns=['Label']).to_csv('train_label.csv', index=False)
pd.DataFrame(test_label,columns=['Label']).to_csv('test_label.csv', index=False)


In [26]:
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

In [28]:
X_train = pd.read_csv('word_seq_train.csv')
X_test = pd.read_csv('word_seq_test.csv')
y_train = pd.read_csv('train_label.csv')
y_test = pd.read_csv('test_label.csv')

In [29]:
X_train = MinMaxScaler().fit_transform(np.array(X_train))
X_test = MinMaxScaler().fit_transform(np.array(X_test))
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [31]:
print(X_train)


[[0.79855441 0.90445903 0.10977162 ...        nan        nan        nan]
 [0.97972159 0.29994644 0.90389123 ...        nan        nan        nan]
 [0.45616383 0.22114355 0.09430045 ...        nan        nan        nan]
 ...
 [0.79855441 0.10792716 0.10327507 ...        nan        nan        nan]
 [0.64375586 0.55095072 0.90389123 ...        nan        nan        nan]
 [0.90322581 0.29090787 0.63994374 ...        nan        nan        nan]]


In [32]:
X_train = pd.read_csv('word_seq_train.csv')
X_test = pd.read_csv('word_seq_test.csv')
y_train = pd.read_csv('train_label.csv')
y_test = pd.read_csv('test_label.csv')

In [33]:
print(X_train)


           0      1        2        3        4        5        6        7  \
0      11948  13512   1655.0   8282.0  10408.0   6479.0  13178.0  13579.0   
1      14655   4483  13512.0   4197.0   9571.0   3345.0  12844.0   7581.0   
2       6832   3306   1424.0  14762.0   7692.0   8675.0   2769.0  14442.0   
3       6708   2899  11249.0   8128.0   7538.0   8243.0  13675.0   8518.0   
4      13512   4958  13958.0  12845.0   1926.0  13512.0   5135.0   9571.0   
...      ...    ...      ...      ...      ...      ...      ...      ...   
10235  13545   1806   1122.0   8013.0   9476.0   9571.0  12286.0   1950.0   
10236   4318   6679   9460.0   2218.0  13512.0  10009.0   9571.0  13512.0   
10237  11948   1615   1558.0  13675.0  12581.0  12120.0  13498.0   9675.0   
10238   9635   8232  13512.0  14232.0   3997.0   5075.0   1624.0   1536.0   
10239  13512   4348   9571.0  14340.0   1360.0   6658.0   1122.0   8543.0   

             8        9  ...  454  455  456  457  458  459  460  461  462  

In [34]:
print (y_train)

       Label
0      False
1       True
2       True
3      False
4       True
...      ...
10235   True
10236   True
10237   True
10238  False
10239  False

[10240 rows x 1 columns]


In [35]:
def padding(seq, maxlen):
    final = []
    for lis in seq:
#         print(lis, '\n***\n')
        # padding
        if len(lis)<maxlen:
            pad = []
            # print(lis)
            for i in range(maxlen-len(lis)):
                if type(lis[0]) == int:
                    pad.append(0)
                else:
                    pad.append([0 for i in range(len(lis[0]))])
            for i in range(len(lis)):
                pad.append(lis[i])
            final.append(pad)
        #truncating
        else:
            trunc = []
            for i in range(maxlen):
                trunc.append(lis[i])
            final.append(trunc)
    return final


In [47]:
def remove_nan(word_seq, data, threshold=0.8):
  df = pd.DataFrame(word_seq)
  cosine_corr = cosine_similarity(df, df)
  for i in range(len(cosine_corr)):
    cosine_corr[i][i] = -1
  idx_cosine_similarity = np.argmax(cosine_corr, axis=1)
  
 
    
  idx = set()
  temp_idx1 = data[data['Statement'].isna()].index

  idx.update(temp_idx1)
 
  
  data.dropna(inplace=True)
  data = data.reset_index(drop=True)
  
  return data, np.delete(word_seq,list(idx),0)

In [40]:
columns = ['Statement','Label']
label_map = { 'FALSE':0, 'TRUE':1}

In [41]:
train = pd.read_csv('train.csv', names=columns)
# print(len(train))#,train.isna().sum())
# train.dropna(inplace=True)
train['Label'] = train['Label'].map(label_map)

In [42]:
train.drop(index=train[train.Statement.isna()].index, inplace=True)
train.drop(index=train[train.Statement==' '].index, inplace=True)
train.drop(index=train[train.Statement=='  '].index, inplace=True)
train.drop(index=train[train.Statement=='\n'].index, inplace=True)
train = train.reset_index()
train.count()

index        10241
Statement    10241
Label        10240
dtype: int64

In [43]:
train_text = np.array(train['Statement'])
test_text = np.array(test['Statement'])

train_tokens = preprocessing(train_text)
test_tokens = preprocessing(test_text)

count = [len(train_tokens[i]) for i in range(len(train_tokens))]
pd.DataFrame(count).describe()

0
count  10241.000000
mean      17.943755
std        9.610519
min        1.000000
25%       12.000000
50%       17.000000
75%       22.000000
max      464.000000

In [45]:
## get vocab

vocab = set()
vocab = getVocab(train_tokens, vocab)
vocab = getVocab(test_tokens, vocab)
vocab = list(vocab)
vocab.sort()
len(vocab)

14959

In [48]:
# words
## convert to vectors 
word_seq_train = wordVec(train_tokens, vocab)
word_seq_test = wordVec(test_tokens, vocab)


## padding
word_seq_train = np.array(padding(word_seq_train, maxlen=50), dtype='float32')
word_seq_test = np.array(padding(word_seq_test, maxlen=50), dtype='float32')


threshold=0.8
# replace nan values in stateinfo, job title with the values from most similar text
train, word_seq_train = remove_nan(word_seq_train, train, threshold)
test, word_seq_test = remove_nan(word_seq_test, test, threshold)


In [49]:
# ## saving as csv
pd.DataFrame(word_seq_train).to_csv('word_seq_train.csv', index=False)
pd.DataFrame(word_seq_test).to_csv('word_seq_test.csv', index=False)

In [54]:
X_train = pd.read_csv('word_seq_train.csv')
X_test = pd.read_csv('word_seq_test.csv')
y_train = pd.read_csv('train_label.csv')
y_test = pd.read_csv('test_label.csv')

X_train = MinMaxScaler().fit_transform(np.array(X_train))
X_test = MinMaxScaler().fit_transform(np.array(X_test))
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)


In [55]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train,y_train)

SVC(kernel='linear')

In [56]:
y_pred_test = clf.predict(X_test)

print('Kernel: linear')


print()
print('Testing')
print(classification_report(y_test,y_pred_test))
plot_cm(y_test,y_pred_test,labels)

Kernel: linear

Testing
              precision    recall  f1-score   support

       False       0.38      0.01      0.01      1169
        True       0.54      0.99      0.70      1382

    accuracy                           0.54      2551
   macro avg       0.46      0.50      0.36      2551
weighted avg       0.47      0.54      0.38      2551



NameError: name 'plot_cm' is not defined